<h1> This notebook is used to analyze the data gathered and cleaned at other notebooks. </h1>

In [59]:
import numpy as np
import pandas as pd
import math

Read the data gathered and cleaned at the data preparation stage.

In [19]:
manhattan = pd.read_csv('manhattan_mean_neighborhood.csv', header=None)
toronto = pd.read_csv('toronto_mean_neighborhood.csv', header=None)

Let's see how the dataframe is.

In [20]:
manhattan.head()

,0,1
0,Accessories Store,0.001210
1,Adult Boutique,0.000605
2,Afghan Restaurant,0.000303
3,African Restaurant,0.000908
4,American Restaurant,0.022995


In [21]:
toronto.head()

,0,1
0,Yoga Studio,0.005882
1,Adult Boutique,0.000588
2,Afghan Restaurant,0.000588
3,Airport,0.000588
4,Airport Food Court,0.000588


Set the venues type as index.

In [22]:
manhattan = manhattan.set_index(0)
toronto = toronto.set_index(0)

In [30]:
manhattan.rename(columns={1:"manhattan"},inplace=True)
toronto.rename(columns={1:"toronto"},inplace=True)

Combine two dataframe into one

In [32]:
combine = pd.concat([manhattan, toronto],axis=1).fillna(0)

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [33]:
combine.head()

,manhattan,toronto
Accessories Store,0.001210,0.000000
Adult Boutique,0.000605,0.000588
Afghan Restaurant,0.000303,0.000588
African Restaurant,0.000908,0.000000
Airport,0.000000,0.000588


Calculate the similarity between two cities using venue type occurency frequency using Jaccard similarity, distance model, pearson correlation, concentration testing.

In [35]:
#Jaccard similarity
intersection_cardinality = len(set.intersection(*[set(manhattan.index), set(toronto.index)]))
union_cardinality = len(set.union(*[set(manhattan.index), set(toronto.index)]))
Jaccardscore = intersection_cardinality/float(union_cardinality)
Jaccardscore

0.5268817204301075

Judging from Jaccard similarity testing, the two cities share around 53% similarity in terms of venue type.
Let's further analyze in the intersection part
<h4>intersection part </h4?

In [43]:
intersect = set.intersection(*[set(manhattan.index), set(toronto.index)])
intersectdata = combine[combine.index.isin(intersect)]
intersectdata.head()

,manhattan,toronto
Adult Boutique,0.000605,0.000588
Afghan Restaurant,0.000303,0.000588
American Restaurant,0.022995,0.017059
Antique Shop,0.000908,0.001765
Arepa Restaurant,0.000908,0.000588


In [53]:
#distance model for similarity
combine['dist'] = combine.manhattan - combine.toronto
combine['distsquare'] = combine.dist * combine.dist
intersectdata['dist'] = intersectdata.manhattan - intersectdata.toronto
intersectdata['distsquare'] = intersectdata.dist * intersectdata.dist

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [56]:
dissim = intersectdata.distsquare.sum() / combine.distsquare.sum()
dissim

0.9378786792645988

Judging from same venue types, the difference between occurency of two places attribute for around 94% of the all difference.
Now we will use correlation to check the similiarity.

In [60]:
fullcor = np.correlate(combine.manhattan, combine.toronto)
intersectcor = np.correlate(intersectdata.manhattan, intersectdata.toronto)
print(fullcor, intersectcor)

[0.01232731] [0.01232731]


Judging from correlation, we can see that despite the venue types share the same majority of categories, the frequency of occurency is quite different for two cities.
Now we will run concentraction test to see any similarity of two cities.

In [66]:
combine.nlargest(10,'manhattan').sum()

manhattan     0.240847
toronto       0.269412
dist         -0.028565
distsquare    0.003859
dtype: float64

In [67]:
combine.nlargest(10,'toronto').sum()

manhattan     0.229652
toronto       0.329412
dist         -0.099760
distsquare    0.004441
dtype: float64

In [68]:
combine.nlargest(20,'manhattan').sum()

manhattan     0.390318
toronto       0.390000
dist          0.000318
distsquare    0.004347
dtype: float64

In [69]:
combine.nlargest(20,'toronto').sum()

manhattan     0.331619
toronto       0.461765
dist         -0.130146
distsquare    0.004704
dtype: float64

In [70]:
combine.nlargest(30,'manhattan').sum()

manhattan     0.499244
toronto       0.474118
dist          0.025126
distsquare    0.004593
dtype: float64

In [71]:
combine.nlargest(30,'toronto').sum()

manhattan     0.419970
toronto       0.556471
dist         -0.136501
distsquare    0.004921
dtype: float64

Judging from the above concentration tests, we can see that the frequency of top 10, 20, 30 venue types' occurency for manhattan is similar to those of toronto, but the opposite is not true. This means that there's significant difference of top venue occurency in Toronto compared to Manhattan.

<h4> Now we will analyze the top 10 venue occured for both cities and see what caused the difference. </h4>

In [81]:
top10man = manhattan.nlargest(10,'manhattan')
top10man

,manhattan
0,
Italian Restaurant,0.041452
Coffee Shop,0.039637
Bakery,0.023601
American Restaurant,0.022995
Café,0.021483
Pizza Place,0.021483
Park,0.018154
Mexican Restaurant,0.017549
French Restaurant,0.017247


In [76]:
top10tor = toronto.nlargest(10, 'toronto')
top10tor

,toronto
0,
Coffee Shop,0.085294
Café,0.055294
Restaurant,0.035294
Italian Restaurant,0.024706
Bakery,0.024118
Bar,0.024118
Hotel,0.024118
Pizza Place,0.020000
Park,0.019412


In [83]:
top10com = pd.concat([top10man, top10tor], axis=1)
top10com

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,manhattan,toronto
American Restaurant,0.022995,0.017059
Bakery,0.023601,0.024118
Bar,NaN,0.024118
Café,0.021483,0.055294
Coffee Shop,0.039637,0.085294
French Restaurant,0.017247,NaN
Gym,0.017247,NaN
Hotel,NaN,0.024118
Italian Restaurant,0.041452,0.024706
Mexican Restaurant,0.017549,NaN


From the top 10 venue types for the two cities (in terms of frequency for the below discussion), we can see that restaurant occupies the most for both cities where Manhattan has more restuarants than Toronto for sure. On the other hand, Toronto has more Bar, Cafe and coffee shop than Manhattan which may indicate Toronto citizens prefer more relaxing acitivies like coffee time or beer time. Manhattan also saw Gym in top 10 venue types which may indicate Manhattan people enjoy exercise more than Toronto people whil